# 1 - Packages

In [ ]:
import time
import random
import numpy as np
import pickle as pkl
import tensorflow as tf

from keras.models import Model, Input
from keras.layers import Embedding, Dense, Dropout, LSTM, CuDNNLSTM, Bidirectional, TimeDistributed
from keras.optimizers import Adam
from keras.utils import Sequence
from keras.initializers import glorot_normal
from keras.callbacks import ModelCheckpoint

In [ ]:
import sys
sys.path.insert(0, '../../../helpers')
from optimizer import NormalizedOptimizer

# 2 - Constants

In [ ]:
WITH_EXTRA_TRAIN = False
DATASET_PATH = '../../../dataset'
CONSTANTS_PATH = '../../../helpers/constants'

with open(CONSTANTS_PATH + '/ARABIC_LETTERS_LIST.pickle', 'rb') as file:
    ARABIC_LETTERS_LIST = pkl.load(file)
with open(CONSTANTS_PATH + '/DIACRITICS_LIST.pickle', 'rb') as file:
    DIACRITICS_LIST = pkl.load(file)
if not WITH_EXTRA_TRAIN:
    with open(CONSTANTS_PATH + '/RNN_SMALL_CHARACTERS_MAPPING.pickle', 'rb') as file:
        CHARACTERS_MAPPING = pkl.load(file)
else:
    with open(CONSTANTS_PATH + '/RNN_BIG_CHARACTERS_MAPPING.pickle', 'rb') as file:
        CHARACTERS_MAPPING = pkl.load(file)
with open(CONSTANTS_PATH + '/RNN_CLASSES_MAPPING.pickle', 'rb') as file:
    CLASSES_MAPPING = pkl.load(file)
with open(CONSTANTS_PATH + '/RNN_REV_CLASSES_MAPPING.pickle', 'rb') as file:
    REV_CLASSES_MAPPING = pkl.load(file)

# 3 - Load Data

In [ ]:
train_raw = None
with open(DATASET_PATH + '/train.txt', 'r') as file:
    train_raw = file.readlines()
if WITH_EXTRA_TRAIN:
    with open(DATASET_PATH + '/extra_train.txt', 'r') as file:
        train_raw += file.readlines()
print('Training examples (raw):', len(train_raw))

val_raw = None
with open(DATASET_PATH + '/val.txt', 'r') as file:
    val_raw = file.readlines()
print('Validation examples (raw):', len(val_raw))

# 4 - Helpers

In [ ]:
def remove_diacritics(data_raw):
    return data_raw.translate(str.maketrans('', '', ''.join(DIACRITICS_LIST)))

In [ ]:
def to_one_hot(data, size):
    one_hot = list()
    for elem in data:
        cur = [0] * size
        cur[elem] = 1
        one_hot.append(cur)
    return one_hot

# 6 - Prepare Data

In [ ]:
def split_data(data_raw):
    data_new = list()
    
    for line in data_raw:
        line = line.replace('.', '.\n')
        line = line.replace(',', ',\n')
        line = line.replace('،', '،\n')
        line = line.replace(':', ':\n')
        line = line.replace(';', ';\n')
        line = line.replace('؛', '؛\n')
        line = line.replace('(', '\n(')
        line = line.replace(')', ')\n')
        line = line.replace('[', '\n[')
        line = line.replace(']', ']\n')
        line = line.replace('{', '\n{')
        line = line.replace('}', '}\n')
        line = line.replace('«', '\n«')
        line = line.replace('»', '»\n')
        
        for sub_line in line.split('\n'):
            if len(remove_diacritics(sub_line).strip()) == 0:
                continue
            
            if len(remove_diacritics(sub_line).strip()) > 0 and len(remove_diacritics(sub_line).strip()) <= 500:
                data_new.append(sub_line.strip())
            else:
                sub_line = sub_line.split()
                tmp_line = ''
                for word in sub_line:
                    if len(remove_diacritics(tmp_line).strip()) + len(remove_diacritics(word).strip()) + 1 > 500:
                        if len(remove_diacritics(tmp_line).strip()) > 0:
                            data_new.append(tmp_line.strip())
                        tmp_line = word
                    else:
                        if tmp_line == '':
                            tmp_line = word
                        else:
                            tmp_line += ' '
                            tmp_line += word
                if len(remove_diacritics(tmp_line).strip()) > 0:
                    data_new.append(tmp_line.strip())

    return data_new

In [ ]:
train_split = split_data(train_raw)
val_split = split_data(val_raw)

In [ ]:
print('Training examples (split):', len(train_split))
print('Validation examples (split):', len(val_split))

In [ ]:
def map_data(data_raw):
    X = list()
    Y = list()
    
    for line in data_raw:        
        x = [CHARACTERS_MAPPING['<SOS>']]
        y = [CLASSES_MAPPING['<SOS>']]
        
        for idx, char in enumerate(line):
            if char in DIACRITICS_LIST:
                continue
            
            x.append(CHARACTERS_MAPPING[char])
            
            if char not in ARABIC_LETTERS_LIST:
                y.append(CLASSES_MAPPING[''])
            else:
                char_diac = ''
                if idx + 1 < len(line) and line[idx + 1] in DIACRITICS_LIST:
                    char_diac = line[idx + 1]
                    if idx + 2 < len(line) and line[idx + 2] in DIACRITICS_LIST and char_diac + line[idx + 2] in CLASSES_MAPPING:
                        char_diac += line[idx + 2]
                    elif idx + 2 < len(line) and line[idx + 2] in DIACRITICS_LIST and line[idx + 2] + char_diac in CLASSES_MAPPING:
                        char_diac = line[idx + 2] + char_diac
                y.append(CLASSES_MAPPING[char_diac])
        
        assert(len(x) == len(y))
        
        x.append(CHARACTERS_MAPPING['<EOS>'])
        y.append(CLASSES_MAPPING['<EOS>'])
        
        y = to_one_hot(y, len(CLASSES_MAPPING))
        
        X.append(x)
        Y.append(y)
    
    X = np.asarray(X)
    Y = np.asarray(Y)
    
    return X, Y

# 7 - Model Structure

In [ ]:
def create_model():
    if tf.test.is_gpu_available():
        SelectedLSTM = CuDNNLSTM
    else:
        SelectedLSTM = LSTM
    
    inputs = Input(shape=(None,))
    
    embeddings = Embedding(input_dim=len(CHARACTERS_MAPPING),
                           output_dim=25,
                           embeddings_initializer=glorot_normal(seed=961))(inputs)
    
    blstm1 = Bidirectional(SelectedLSTM(units=256,
                                     return_sequences=True,
                                     kernel_initializer=glorot_normal(seed=961)))(embeddings)
    dropout1 = Dropout(0.5)(blstm1)
    blstm2 = Bidirectional(SelectedLSTM(units=256,
                                     return_sequences=True,
                                     kernel_initializer=glorot_normal(seed=961)))(dropout1)
    dropout2 = Dropout(0.5)(blstm2)
    
    dense1 = TimeDistributed(Dense(units=512,
                                   activation='relu',
                                   kernel_initializer=glorot_normal(seed=961)))(dropout2)
    dense2 = TimeDistributed(Dense(units=512,
                                   activation='relu',
                                   kernel_initializer=glorot_normal(seed=961)))(dense1)
    
    output = TimeDistributed(Dense(units=len(CLASSES_MAPPING),
                                   activation='softmax',
                                   kernel_initializer=glorot_normal(seed=961)))(dense2)
    
    model = Model(inputs, output)
    
    model.compile(loss='categorical_crossentropy', optimizer=NormalizedOptimizer(Adam()))
    
    return model

In [ ]:
model = create_model()
model.summary()

# 8 - Training

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, lines, batch_size):
        self.lines = lines
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.lines) / float(self.batch_size)))

    def __getitem__(self, idx):
        lines = self.lines[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch, Y_batch = map_data(lines)
        
        X_max_seq_len = np.max([len(x) for x in X_batch])
        Y_max_seq_len = np.max([len(y) for y in Y_batch])
        
        assert(X_max_seq_len == Y_max_seq_len)
        
        X = list()
        for x in X_batch:
            x = list(x)
            x.extend([CHARACTERS_MAPPING['<PAD>']] * (X_max_seq_len - len(x)))
            X.append(np.asarray(x))
        
        Y_tmp = list()
        for y in Y_batch:
            y_new = list(y)
            y_new.extend(to_one_hot([CLASSES_MAPPING['<PAD>']] * (Y_max_seq_len - len(y)), len(CLASSES_MAPPING)))
            Y_tmp.append(np.asarray(y_new))
        Y_batch = Y_tmp
        
        Y_batch = np.asarray(Y_batch)
        
        return np.asarray(X), Y_batch

In [ ]:
def fit_model(model, epochs, batch_size, train_split, val_split):
    random.shuffle(train_split)
    train_split = list(sorted(train_split, key=lambda line: len(remove_diacritics(line))))
    random.shuffle(val_split)
    val_split = list(sorted(val_split, key=lambda line: len(remove_diacritics(line))))
        
    checkpoint_path = 'checkpoints/epoch{epoch:02d}.ckpt'
    checkpoint_cb = ModelCheckpoint(checkpoint_path, verbose=0)
    
    training_generator = DataGenerator(train_split, batch_size)
    val_generator = DataGenerator(val_split, batch_size)

    model.fit_generator(generator=training_generator,
                        validation_data=val_generator,
                        epochs=epochs,
                        callbacks=[checkpoint_cb])

In [ ]:
start_time = time.time()
fit_model(model, 50, 256, train_split, val_split)
end_time = time.time()
print('--- %s seconds ---' % round(end_time - start_time, 2))

# 9 - Evaluation

In [ ]:
def predict(line, model):
    X, _ = map_data([line])
    predictions = model.predict(X).squeeze()
    predictions = predictions[1:]
    
    output = ''
    for char, prediction in zip(remove_diacritics(line), predictions):
        output += char
        
        if char not in ARABIC_LETTERS_LIST:
            continue
        
        if '<' in REV_CLASSES_MAPPING[np.argmax(prediction)]:
            continue

        output += REV_CLASSES_MAPPING[np.argmax(prediction)]

    return output

In [ ]:
print(predict('مرحبا كيف الحال يا صديقي العزيز', model))

In [ ]:
with open(DATASET_PATH + '/test.txt', 'r') as file:
    lines = file.readlines()

In [ ]:
results = list()
for idx, line in enumerate(lines):
    line = remove_diacritics(line.strip())
    x = predict(line, model)
    results.append(x)

In [ ]:
with open('outputs.txt', 'w') as file:
    file.write('\n'.join(results))